# Anna's Archive SQL → DataFrame

Explore the Anna's Archive Spotify SQLite database and load it as a pandas DataFrame.

In [1]:
import sqlite3
import pandas as pd
import os

# Path to Anna's Archive DB (adjust if needed)
DB_PATHS = [
    "annas_archive_data/spotify_clean_audio_features.sqlite3",
    "annas_archive_data/spotify_clean_track_files.sqlite3",
    os.path.expanduser("~/Downloads/annas_archive_spotify_2025_07_metadata/spotify_clean_audio_features.sqlite3"),
]

db_path = None
for p in DB_PATHS:
    if os.path.exists(p):
        db_path = p
        break

if db_path is None:
    raise FileNotFoundError("Anna's Archive DB not found. Tried:" + str(DB_PATHS))

print(f"Using: {db_path}")

Using: annas_archive_data/spotify_clean_audio_features.sqlite3


In [2]:
# Inspect schema
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = [r[0] for r in cursor.fetchall()]
print("Tables:", tables)

Tables: ['track_audio_features']


In [3]:
# Get schema for track_audio_features (or first table)
table = "track_audio_features" if "track_audio_features" in tables else tables[0]
cursor.execute(f"PRAGMA table_info({table})")
columns = cursor.fetchall()
print(f"Columns in {table}:")
for c in columns:
    print(f"  {c[1]} ({c[2]})")

Columns in track_audio_features:
  rowid (INTEGER)
  track_id (TEXT)
  fetched_at (INTEGER)
  null_response (INTEGER)
  duration_ms (INTEGER)
  time_signature (INTEGER)
  tempo (INTEGER)
  key (INTEGER)
  mode (INTEGER)
  danceability (REAL)
  energy (REAL)
  loudness (REAL)
  speechiness (REAL)
  acousticness (REAL)
  instrumentalness (REAL)
  liveness (REAL)
  valence (REAL)


In [4]:
# Load into DataFrame
# Adjust limit and WHERE clause as needed for exploration
query = f"""
SELECT *
FROM {table}
WHERE null_response = 0
  AND tempo IS NOT NULL
  AND tempo > 0
LIMIT 100000
"""

df = pd.read_sql_query(query, conn)
conn.close()

print(f"Shape: {df.shape}")
df.head(10)

Shape: (100000, 17)


,rowid,track_id,fetched_at,null_response,duration_ms,time_signature,tempo,key,mode,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence
0,1,2Pe9cbhOTvOUTDE4bl7zzl,1755734400000,0,630506,4,87.683,6,0,0.279,0.391,-12.054,0.3200,0.816,0.737000,0.1770,0.0299
1,2,0wP732NKm8XgXu78XLRWoR,1755734400000,0,97216,4,105.298,5,1,0.429,0.318,-11.685,0.0566,0.587,0.782000,0.2020,0.3600
2,3,22L6EJdnjx8oIo7GiF9hLe,1755734400000,0,75180,4,117.657,0,1,0.283,0.581,-9.420,0.0555,0.923,0.939000,0.1060,0.0362
3,4,3a519lgQ13JXNi0G73mwMT,1755734400000,0,149447,4,100.685,5,0,0.244,0.995,-0.690,0.1250,0.780,0.799000,0.1320,0.0634
4,5,27yP7p2lxWYTtnldRN8Kzx,1755734400000,0,120816,4,123.499,7,1,0.313,0.618,0.411,0.0730,0.843,0.109000,0.1260,0.1870
5,6,0NbgRl9ysfWbkQkvHI0ZTM,1755734400000,0,168986,4,117.223,0,0,0.759,0.256,-10.212,0.0456,0.685,0.000119,0.1020,0.0989
6,7,1J6qdeHnI3a6IWkXzGEwH4,1755734400000,0,95791,4,101.602,5,0,0.883,0.640,-0.878,0.0849,0.242,0.000036,0.0890,0.3670
7,8,5D8e5V1hNhCJPJUovKZbDU,1755734400000,0,173793,3,76.552,7,1,0.325,0.691,-0.127,0.1680,0.784,0.029600,0.3730,0.0413
8,9,5vfGvoiG2Y8Xy0cZTXB4jF,1755734400000,0,265967,4,137.822,11,0,0.332,0.313,-15.958,0.0407,0.955,0.894000,0.0998,0.0390
9,10,3mW5dgtVIoOliiZ7c2sNIO,1755734400000,0,215786,4,131.958,3,0,0.373,0.373,-12.198,0.0368,0.897,0.792000,0.1840,0.1560


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 17 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   rowid             100000 non-null  int64  
 1   track_id          100000 non-null  object 
 2   fetched_at        100000 non-null  int64  
 3   null_response     100000 non-null  int64  
 4   duration_ms       100000 non-null  int64  
 5   time_signature    100000 non-null  int64  
 6   tempo             100000 non-null  float64
 7   key               100000 non-null  int64  
 8   mode              100000 non-null  int64  
 9   danceability      100000 non-null  float64
 10  energy            100000 non-null  float64
 11  loudness          100000 non-null  float64
 12  speechiness       100000 non-null  float64
 13  acousticness      100000 non-null  float64
 14  instrumentalness  100000 non-null  float64
 15  liveness          100000 non-null  float64
 16  valence           100

In [6]:
df.describe()

,rowid,fetched_at,null_response,duration_ms,time_signature,tempo,key,mode,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence
count,100000.000000,1.000000e+05,100000.0,1.000000e+05,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,50107.721820,1.755734e+12,0.0,2.048350e+05,3.894940,122.121287,5.249150,0.615220,0.583360,0.551470,-11.255309,0.127616,0.320136,0.339531,0.188468,0.449149
std,28926.283875,0.000000e+00,0.0,1.421561e+05,0.477016,30.319648,3.618025,0.486546,0.187006,0.264185,6.088131,0.182860,0.348263,0.404667,0.151960,0.266863
min,1.000000,1.755734e+12,0.0,1.500000e+04,0.000000,30.623000,0.000000,0.000000,0.000000,0.000020,-52.023000,0.022400,0.000000,0.000000,0.009190,0.000000
25%,25063.750000,1.755734e+12,0.0,1.440502e+05,4.000000,98.860250,2.000000,0.000000,0.463000,0.348000,-13.941250,0.038000,0.010400,0.000002,0.098200,0.224000
50%,50110.500000,1.755734e+12,0.0,1.921430e+05,4.000000,121.642000,5.000000,1.000000,0.608000,0.561000,-9.828000,0.055000,0.153000,0.015000,0.124000,0.434000
75%,75163.250000,1.755734e+12,0.0,2.400000e+05,4.000000,140.055000,9.000000,1.000000,0.725000,0.769000,-7.083000,0.119000,0.626000,0.832000,0.236000,0.658000
max,100202.000000,1.755734e+12,0.0,6.068419e+06,5.000000,245.999000,11.000000,1.000000,0.994000,1.000000,5.349000,0.969000,0.996000,1.000000,0.994000,1.000000


## Spotify API: Genre & Popularity

For each track ID, we fetch:
- **Popularity** from [Get Several Tracks](https://developer.spotify.com/documentation/web-api/reference/get-several-tracks)
- **Genres** from [Get Several Artists](https://developer.spotify.com/documentation/web-api/reference/get-several-artists) (tracks don't have genres; artists do)

Then: sort by genre → popularity within each genre. Multi-genre tracks are counted in every genre they belong to.

In [7]:
# Spotify API setup (needs SPOTIPY_CLIENT_ID, SPOTIPY_CLIENT_SECRET in .env)
from dotenv import load_dotenv
load_dotenv()

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials())

In [ ]:
def fetch_genre_and_popularity(track_ids, batch_size=50, delay=0.1):
    """
    Fetch popularity (from track) and genres (from main artist) for each track_id.
    Returns list of dicts: {track_id, popularity, genres, artist_id}.
    """
    results = []
    for i in range(0, len(track_ids), batch_size):
        batch = track_ids[i:i+batch_size]
        tracks = sp.tracks(batch)["tracks"]
        artist_ids = set()
        track_info = {}  # track_id -> {popularity, artist_id}

        for t in tracks:
            if not t:
                continue
            tid = t["id"]
            popularity = t.get("popularity", 0)
            artists = t.get("artists", [])
            main_artist_id = artists[0]["id"] if artists else None
            track_info[tid] = {"popularity": popularity, "artist_id": main_artist_id}
            if main_artist_id:
                artist_ids.add(main_artist_id)

        # Fetch artist genres
        artist_to_genres = {}
        if artist_ids:
            aid_list = list(artist_ids)
            for j in range(0, len(aid_list), 50):
                arts = sp.artists(aid_list[j:j+50])["artists"]
                for a in arts:
                    if a and a.get("id"):
                        artist_to_genres[a["id"]] = a.get("genres", []) or []

        for tid, info in track_info.items():
            genres = artist_to_genres.get(info["artist_id"], []) if info["artist_id"] else []
            results.append({
                "track_id": tid,
                "popularity": info["popularity"],
                "genres": genres,
                "artist_id": info["artist_id"],
            })

        time.sleep(delay)

    return results

In [ ]:
# Sample track IDs to fetch (start small for exploration; increase later)
SAMPLE_SIZE = 1000  # adjust as needed
track_ids = df["track_id"].head(SAMPLE_SIZE).tolist()

meta = fetch_genre_and_popularity(track_ids)
df_meta = pd.DataFrame(meta)
df_meta.head(10)

Your application has reached a rate/request limit. Retry will occur after: 9756 s


In [ ]:
# Expand: one row per (track_id, genre) so multi-genre tracks count in all genres
rows = []
for _, r in df_meta.iterrows():
    genres = r["genres"] if isinstance(r["genres"], list) else (r["genres"] or [])
    if not genres:
        genres = ["(no genre)"]
    for g in genres:
        rows.append({"track_id": r["track_id"], "genre": g, "popularity": r["popularity"]})

df_expanded = pd.DataFrame(rows)

In [ ]:
# Genre stats: count and % (multi-genre tracks counted in each genre)
total_unique_tracks = df_meta["track_id"].nunique()
genre_counts = df_expanded.groupby("genre").agg(
    song_count=("track_id", "nunique"),  # unique tracks per genre
).sort_values("song_count", ascending=False)

genre_counts["pct_of_database"] = (genre_counts["song_count"] / total_unique_tracks * 100).round(2)
genre_counts

,song_count,pct_of_database
genre,,
(no genre),755,75.5
underground hip hop,48,4.8
hardcore,25,2.5
epadunk,22,2.2
death metal,20,2.0
...,...,...
latin house,1,0.1
latin indie,1,0.1
minimal techno,1,0.1


In [ ]:
# Sort: by genre, then by popularity (desc) within each genre
df_sorted = df_expanded.sort_values(["genre", "popularity"], ascending=[True, False])
df_sorted.head(20)

,track_id,genre,popularity
976,2lIzIXl3jcnvxgZxpmuwZ4,(no genre),46
896,2CLp3mcgjTdBVEzUtXnnn1,(no genre),26
936,0XyzNlQs5wWLMfeUSAwGD6,(no genre),25
883,2eyBlHT7EoJLBA5FnVYq6j,(no genre),24
890,0GmQAtTWjhfRdHud7JgoVS,(no genre),24
934,6UKL24dDP8doCngfEyXvTE,(no genre),24
935,6ejUQG2EDIBkX32sr6Abwd,(no genre),24
555,4PFx9vhlWnpB4Qcgr3AR4O,(no genre),23
772,1jq9oDICJs5yXOIdEuYnpP,(no genre),23
876,6UcCvg9WroKcIo1dOoeQV4,(no genre),23


In [ ]:
# Summary
print(f"Total unique tracks: {total_unique_tracks}")
print(f"Unique genres: {genre_counts.shape[0]}")
print(f"\nTop 15 genres by song count:")
print(genre_counts.head(15).to_string())

Total unique tracks: 1000
Unique genres: 63

Top 15 genres by song count:
                     song_count  pct_of_database
genre                                           
(no genre)                  755             75.5
underground hip hop          48              4.8
hardcore                     25              2.5
epadunk                      22              2.2
death metal                  20              2.0
rock en español              18              1.8
jangle pop                   12              1.2
screamo                      12              1.2
slowcore                     11              1.1
grindcore                     9              0.9
french jazz                   8              0.8
tech house                    8              0.8
disco house                   7              0.7
house                         7              0.7
doom metal                    6              0.6


In [ ]:
# Optional: full df with genre + popularity (one row per track, genres as list)
df_full = df[df["track_id"].isin(df_meta["track_id"])].merge(
    df_meta[["track_id", "popularity", "genres"]], on="track_id", how="left"
)
df_full = df_full.sort_values(["popularity"], ascending=False)  # overall by popularity
df_full.head()

,rowid,track_id,fetched_at,null_response,duration_ms,time_signature,tempo,key,mode,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,popularity,genres
888,897,2lIzIXl3jcnvxgZxpmuwZ4,1755734400000,0,216000,4,89.970,10,1,0.554,0.215,-18.450,0.0309,0.9000,0.0001,0.105,0.456,46,[]
320,328,0BMZSSz1Ndwsxaebq6Sd49,1755734400000,0,200813,4,178.215,6,0,0.463,0.838,-7.000,0.0462,0.1770,0.0000,0.124,0.730,40,[rock en español]
471,479,6MPmTzi1FdDKu3yhfhVd1Y,1755734400000,0,163213,4,83.008,5,1,0.567,0.417,-7.152,0.0325,0.6170,0.0000,0.108,0.360,36,[latin pop]
321,329,5Bx7dDEWKlQBHaB2fwr3Lq,1755734400000,0,249360,4,130.840,7,1,0.580,0.691,-7.161,0.0299,0.0707,0.0000,0.218,0.655,32,[rock en español]
469,477,1U4xYstX4dcJbjlGZ3sQy9,1755734400000,0,176720,4,104.934,11,1,0.748,0.752,-5.802,0.1970,0.0184,0.0000,0.357,0.592,32,[latin pop]
